In [264]:
import boto3
import pandas as pd
import json

import load_data as ld
import helpers as h

### Create Resource and load bucket file names

In [205]:
s3_resource = boto3.resource('s3')

bucket_name = 'leafliink-data-interview-exercise'
bucket = s3_resource.Bucket(bucket_name)
keys = h.get_keys(bucket)


len(keys)

28953

### Parse out dirs and prefixes

In [186]:
keys_df = pd.DataFrame(keys, columns=['original'])
dir_files = keys_df['original'].str.split('/',  n = 1, expand = True)
prefix_file = dir_files[1].str.split('_', n=1, expand=True)
keys_df['dir'], keys_df['file'], keys_df['prefix'], keys_df['detail'] = dir_files[0], dir_files[1], prefix_file[0], prefix_file[1]

keys_df.head()

### File Structure Anaylsis and Summary

In [95]:
dirs = list(keys_df.dir.drop_duplicates())
file_types = list(keys_df.prefix.drop_duplicates())
summary = keys_df[['dir','prefix', 'original']].groupby(['dir', 'prefix']).count().reset_index()
first_val = keys_df[['dir','prefix', 'original']].groupby(['dir', 'prefix']).first().reset_index()
first_val.columns = ['dir','prefix', 'first']
summary = pd.concat([summary, first_val['first']], axis=1)
summary = summary[~(summary.prefix=='')]

print('Directories: ', dirs)
print('Prefixes (filetype): ', file_types)
summary

Directories:  ['01', '02', '03', '04', '05', '06', '07']
Prefixes (filetype):  ['', 'clicks', 'impressions']


,dir,prefix,original,first
1,01,clicks,66,01/clicks_2020_02_01_0011_10417_engine-i-0c872...
2,01,impressions,2558,01/impressions_2020_02_01_0001_11392_engine-i-...
4,02,clicks,53,02/clicks_2020_02_02_0044_13471_engine-i-05fcb...
5,02,impressions,2336,02/impressions_2020_02_02_0000_10417_engine-i-...
7,03,clicks,166,03/clicks_2020_02_03_0019_12062_engine-i-0a4cf...
8,03,impressions,5083,03/impressions_2020_02_03_0000_10417_engine-i-...
10,04,clicks,160,04/clicks_2020_02_04_0015_10417_engine-i-0c872...
11,04,impressions,5089,04/impressions_2020_02_04_0000_10417_engine-i-...
13,05,clicks,114,05/clicks_2020_02_05_0000_10417_engine-i-0c872...
14,05,impressions,4365,05/impressions_2020_02_05_0000_10417_engine-i-...


Note: Dirs correspond to month
### Download Test Files

In [140]:
test_files = list(summary['first'])
local_files = [f'test_files/{file.split("/")[1]}' for file in test_files]
for i,file in enumerate(test_files):
    s3_resource.Object(bucket_name, file).download_file(local_files[i])

### Read JSON files
- some files error because of following structure: '{}/n{}/n'
- exception catches and deals with these

In [236]:
json_files = ld.read_jsons(local_files)

### Flatten fields 
- User and Device store another dict
- Append info (i.e. User_Key)

In [231]:
final_test = [h.flatten_dict(file) for file in json_files]
sample_df = pd.DataFrame(final_test)
sample_df.columns = [h.format_column(col.replace(':','_')) for col in sample_df.columns]
sample_df.to_csv('test_files/test.csv')

### Below we see table sample
- Note: impressions and clicks distinguished by existance of impression_id (and other fields) 
- In future state would do dufther cleaning (i.e. date field)

In [232]:
pd.set_option('display.max_columns', 500)
sample_df

,meta_schema,meta_version,gdpr_computed,gdpr_source,remote_i_p,user_agent,ecpm,datacenter,burn_in,is_valid_u_a,user__key,user__is_new,user_key,click_count,id,created_on,event_created_on,impression_created_on,ad_type_id,brand_id,campaign_id,categories,channel_id,creative_id,creative_pass_id,delivery_mode,first_channel_id,impression_id,decision_id,is_no_track,is_tracking_cookie_events,keywords,device_brand_name,device_model_name,device_os_raw_version,device_os_major_version,device_os_minor_version,device_browser,device_browser_raw_version,device_browser_major_version,device_browser_minor_version,device_form_factor,matching_keywords,network_id,pass_id,phantom_creative_pass_id,placement_name,phantom_pass_id,price,priority_id,rate_type,revenue,served_by,served_by_pid,served_by_asg,site_id,url,zone_id,impression_count,decision_idx,auction_bids,is_publisher_payout_exempt,relevancy_score,net_revenue,gross_revenue
0,event,1.3,True,none,0.0.0.0,,0,False,False,True,19288,False,19288,1.0,92555ce29d69470da20623c1be8178d7,/Date(1580515872267)/,1580515883978,1580515872267,16,545939,1316050,[],41720,16384669,19046021,3,41720,a3fa3f4e90a34a969c02d6a7c9b7a2b7,6f9e3f3b6b1d4b1293f9f9551b74d44a,False,True,"featured_brand,featured_brand,featured_brand,f...",Google,Chrome,0,0,0,Chrome Desktop,79.0,79,0,desktop,[],10340,11030539,0,shop-brands-featured-brand-4,0,479.2,167186,1,0,i-085444ceb20135132,12659,bifrost-adservers-default-us-east-1,1096373,https://leaflink.com/c/pakalolo-supply-company...,211622,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,event,1.3,True,none,0.0.0.0,,0,False,False,True,9496,False,9496,NaN,2976a7acbd2342babb273795dd9293ce,/Date(1580515294389)/,1580515295233,1580515294389,2480,542240,1310070,[],41720,16347236,19020242,3,41720,NaN,12a2999139e041a4a13f0412693bef7f,False,True,"discover_products_banner,discover_products_banner",Google,Chrome,0,0,0,Chrome Desktop,79.0,79,0,desktop,[],10340,11015174,0,discovery-right,0,439.5,178494,1,0,i-0b2e383c3275ffd63,119576,bifrost-adservers-default-us-east-1,1096373,https://leaflink.com/shop/,211626,1.0,0.0,1.0,False,500.0,0.0,0.0
2,event,1.3,True,none,0.0.0.0,,0,False,False,True,18971,False,18971,1.0,ee4e154fc84548869b953de028d8b5c1,/Date(1580604274705)/,1580604276624,1580604274705,2480,543270,1326346,[],41720,16434153,19097417,3,41720,5b3c9f02f72f49b2b33136586fca8165,5f93c4c2eaf44c84a32b8b601d364bca,False,True,"discover_products_banner,discover_products_banner",Google,Chrome,0,0,0,Chrome Desktop,79.0,79,0,desktop,[],10340,11063345,0,discovery-left,0,750,167186,1,0,i-097ec0b626e835dfd,28634,bifrost-adservers-default-us-east-1,1096373,https://leaflink.com/shop/,211625,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,event,1.3,True,none,0.0.0.0,,0,False,False,True,16180,False,16180,NaN,6be3815e0869400c9b438d68ee23f9ef,/Date(1580601638847)/,1580601639839,1580601638847,16,552445,1326893,[],41720,16438062,19102611,3,41720,NaN,317c3e7e29714bad9056de5b9e232a35,False,True,"featured_brand,featured_brand,featured_brand,f...",Google,Chrome,0,0,0,Chrome Desktop,79.0,79,0,desktop,[],10340,11066425,0,shop-brands-featured-brand-4,0,0,167186,1,0,i-0e4532675b98925e1,29060,bifrost-adservers-default-us-east-1,1096373,https://leaflink.com/c/the-kush-spot/shop/brands/,211622,1.0,0.0,1.0,False,500.0,0.0,0.0
4,event,1.3,True,none,0.0.0.0,,0,False,False,True,14403,False,14403,1.0,4c65d2001e784fdd9400876688161a57,/Date(1580689150958)/,1580689191422,1580689150958,2480,543275,1326369,[],41720,16434256,19097585,3,41720,1343fbd9388846fbb914ad072bd5df1c,0643c627eb4e4b23bb8aea45c58fc386,False,True,"discover_products_banner,discover_products_banner",Google,Chrome,0,0,0,Chrome Desktop,79.0,79,0,desktop,[],10340,11063421,0,discovery-right,0,425,167186,1,0,i-0d564775f53b4dfe7,56768,bifrost-adservers-default-us-east-1,1096373,https://leaflink.com/shop/,211626,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,event,1.3,True,none,0.0.0.0,,0,False,False,True,7463,False,7463,1.0,6ce6b70c5a95461c9e89de92e7e13f9e,/Date(1580775302160)/,1580775316853,1580775302160,16,543271,1326357,[],4172